# Create Satellite Image Tiles 

This notebook is used to crop image tiles from a specified satellite image stack at DHS locations that are contained wiythin the image stack. The expected input is a single image stack that covers an entire AOI or a region of an AOI. When acquiring satellite imagery it is not practyical to download a single file for an entire AOI, but multiple files can later be stiched together into a single GeoTiff file and this notebook can be executed just once to certate all the image tiles, or alternatievley, this notebook can be execute multiple times, once for each non-overlapping image stack, where each execution crops image tiles for provided image stack.


## Input
The expected input is a single image stack that covers an entire AOI or a region of an AOI.
<pre style="font-family: monospace;">
./GIS-Image-Stack-Processing
    /AOI/
        PK/
            Image_Tiles/
                    :
            Satellite_Stack/
                    <span style="color: blue;">PK_aoi_stack_3band.tif</span>  RGB example
                    </pre>

## Output 
The output satellite tiles (for each DHS location) stored in `Satellite_Tiles` within the hierarchy below. 
<pre style="font-family: monospace;">
./GIS-Image-Stack-Processing
    /AOI/
        PK/
            Image_Tiles/
                    :
            Satellite_Tiles/
                PK_1_C-1_30m.tif
                PK_2_C-2_30m.tif
                    :
                PK_560_C-580_30m.tif
                </pre>

In [22]:
import os
import sys
import shutil
import glob as glb
from osgeo import gdal
from dataclasses import dataclass

cache_dir = 'project_utils/__pycache__'
if os.path.exists(cache_dir):
    shutil.rmtree(cache_dir)
    
sys.path.append('../GIS-Image-Stack-Processing') 

# Import module that contains several convenience functions (e.g., gdal wrappers)
from project_utils import *

#----------------------------------------------------------------------------------------
# *** IMPORTANT: SYSTEM PATH TO SET ***
#----------------------------------------------------------------------------------------
# The following path is required, as it contains GDAL binaries used for several 
# pre-processing functions. The pathname corresponds to the Conda virtual environment 
# created for this project (e.g., "py39-pt").
#
# Note: GDAL was adopted as a benchmark to compare the original GIS data produced by 
# another team. However, similar functionality could be implemented using the Rasterio 
# Python package. If Rasterio is used, it would eliminate the need for GDAL binaries 
# and this system path specification.
#----------------------------------------------------------------------------------------

os.environ['PATH'] += ':/Users/billk/miniforge3/envs/py39-pt/bin/' 

## 1 Specify the Country Code for the AOI

In [2]:
#-------------------------------------------------
# REQUIRED CONFIGURATIONS HERE
#-------------------------------------------------
country_code = 'PK'   # Set the country code
#-------------------------------------------------

In [3]:
shapefile_path = aoi_configurations[country_code]['shapefile']

# The new base path that you want to prepend
base_path = "../GIS-Image-Stack-Processing/"

# Prepend the base path to the shapefile path
shapefile_path = os.path.join(base_path, shapefile_path)

crs_lat = aoi_configurations[country_code]['crs_lat']
crs_lon = aoi_configurations[country_code]['crs_lon']

#------------------------------------------------------------------------------------------------------------
# A Lambert-Azmuthal Equal Area (LAEA) projectoin CRS is used that requires the definition of a CRS 
# orgign (crs_lat, crs_lon). Each AOI defined in the aoi_configurations.py module contains these coordinates.
#------------------------------------------------------------------------------------------------------------
expected_crs = f'+proj=laea +lat_0={crs_lat} +lon_0={crs_lon} +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs'

case = country_code
print(shapefile_path)

../GIS-Image-Stack-Processing/DHS/PK_2017-18_DHS/PKGE71FL/PKGE71FL.shp


In [4]:
# Shape file fields
cluster_field  = 'DHSCLUST'
lat_field      = 'LATNUM'
lon_field      = 'LONGNUM'

# Set the resolution for programmatic file naming below
res = 30

expected_pixel_size  = (res, res)    # This should match the pixel size in the input rasters

# Build a list of the raster (produced by: prep_geospatial_data.ipynb)
aoi_image_stack_folder = f'../GIS-Image-Stack-Processing/AOI/{country_code}/Satellite_Stack/'

# Assumes a single AOI image stack that covers the DHS survey locations.
# aoi_image_stack_path = f'{aoi_image_stack_folder}{country_code}_aoi_stack_6band.tif'
aoi_image_stack_path = f'{aoi_image_stack_folder}{country_code}_aoi_stack_3band.tif'

image_tile_folder = f'../GIS-Image-Stack-Processing/AOI/{country_code}/Satellite_Tiles/'

image_tile_suffix = f'{res}m'

vrt_file_suffix = f'{res}m'

print(aoi_image_stack_path)

../GIS-Image-Stack-Processing/AOI/PK/Satellite_Stack/PK_aoi_stack_3band.tif


In [5]:
print(aoi_image_stack_path)
print('\n')
print(image_tile_folder)
print(vrt_file_suffix)

../GIS-Image-Stack-Processing/AOI/PK/Satellite_Stack/PK_aoi_stack_3band.tif


../GIS-Image-Stack-Processing/AOI/PK/Satellite_Tiles/
30m


In [6]:
# run_gdalinfo(f'../GIS-Image-Stack-Processing/AOI/{country_code}/Satellite_Stack/{country_code}_aoi_stack_6band.tif')

## 2 Extract Cluster IDs and Associated GPS Coordinates

In [7]:
if os.path.exists(shapefile_path):
    print("File exists.")
else:
    print("File does not exist.")

File exists.


## 3 Load AOI Raster Data

In [8]:
print(aoi_image_stack_path)

../GIS-Image-Stack-Processing/AOI/PK/Satellite_Stack/PK_aoi_stack_3band.tif


In [9]:

raster, crs_match, pixel_size_match = load_raster(aoi_image_stack_path, expected_crs, expected_pixel_size)
result = {
        'path': aoi_image_stack_path,
        'raster': raster, 
        'crs_match': crs_match,
        'pixel_size_match': pixel_size_match
}

## 4 Project the DHS GPS Locations to the Image Stack CRS 

Some DHS cluster IDs contain bogos coordinates and therefore such clusters should be excluded. 

In [10]:
cluster_data, erroneous_cluster_ids = extract_cluster_data(shapefile_path, cluster_field, lat_field, lon_field)

Erroneous clusters detected:
Cluster ID: 535, Latitude: 0.0, Longitude: 0.0


/Users/billk/dev/BMGF/Deep-Learning-Global-Health-Analytics/Satellite_Data/../GIS-Image-Stack-Processing/project_utils/gist_utils.py:796: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cluster_data[cluster_field] = cluster_data[cluster_field].astype(float).astype(int)


In [11]:
print(erroneous_cluster_ids)

[535]


### Optional Data Inspection

In [12]:
# Print data for a specific cluster
cluster_id = 1
indices = [cluster_id]
for index in indices:
    # Using .iloc for positional access
    row = cluster_data.iloc[index]
    cluster_id, x, y = row[cluster_field], row[lat_field], row[lon_field]

    print(f"({cluster_id}, {x:.2f}, {y:.2f})")

(2.0, 35.89, 71.73)


In [13]:
# Print the first few elements of cluster_data to understand its structure
for item in cluster_data[:10]:  # Adjust the slicing as necessary for large datasets
    print(item)


DHSCLUST
LATNUM
LONGNUM


In [14]:
# Initialize a flag to check if all CRS matches and a list to collect mismatched CRS details
all_crs_match = True
mismatched_details = []

if not result['crs_match']:
    all_crs_match = False
    # Collecting detailed information about the mismatch, including the file path and the actual CRS
    mismatched_details.append(f"Path: {result['path']}, Raster CRS: {result['raster'].crs}")

if all_crs_match:
    
    # Assuming cluster_data is a DataFrame returned from extract_cluster_data
    cluster_data_tuples = list(cluster_data.to_records(index=False))

    crs_coordinates = convert_cluster_coordinates(cluster_data_tuples, src_crs='EPSG:4326', dst_crs=expected_crs)
    print(len(crs_coordinates))
        
else:
    print("*** Error: CRS does not match.")
    for detail in mismatched_details:
        print(detail)

560


## 5 Find DHS Points that Fall within the Input Satellite Image Stack
The input satellite image stack was created from multiple satellite granule 

In [15]:
all_pixel_match = result['pixel_size_match']

# if all_pixel_match:
if True:
   
    # Collect number of points in each dataset for comparison
    number_of_points_per_dataset = []


    raster = result['raster']
        
    points_within_raster = find_points_within_raster(raster, crs_coordinates, expected_crs)
        
    # Store the points within raster into the results dictionary for each raster
    result['points_within_raster'] = points_within_raster
    number_of_points_per_dataset.append(len(points_within_raster))
    print(f"Points w/in bounds: {result['path']}: {len(points_within_raster)}\n")

    # Check if all datasets have the same number of points
    if len(set(number_of_points_per_dataset)) == 1:
        print("All datasets have the same number of points.")
    else:
        print("Warning: Datasets have varying numbers of points. Here are the counts per dataset:", number_of_points_per_dataset)

else:
    print("Pixel size match does not match for one or more rasters.")


Points w/in bounds: ../GIS-Image-Stack-Processing/AOI/PK/Satellite_Stack/PK_aoi_stack_3band.tif: 59

All datasets have the same number of points.


## 6 Crop Image Tiles from AOI Image Stack
Loop over each data type, stored in memory as a raster, and crop an image tile of the specified size for each of the survey points in the `points_within_raster list`. Additionally, build a VRT file that references the image tiles for each data type. The VRT facilitates loading a large number of image tiles in QGIS for visualization purposes.

In [16]:
def build_vrt(image_tile_folder, vrt_file):
   
    # Get a list of all .tif files in the directory
    tif_files = glb.glob(os.path.join(image_tile_folder, "*.tif"))

    # Create a new VRT dataset
    vrt_options = gdal.BuildVRTOptions(VRTNodata=-999)
    vrt = gdal.BuildVRT(vrt_file, tif_files, options=vrt_options)

    # Check if the VRT dataset was created successfully
    if vrt is None:
        print("Failed to build VRT")
    else:
        vrt.FlushCache()  # Write to disk
        print(f"VRT built successfully at {vrt_file}")

In [17]:
print(image_tile_folder)
print(image_tile_suffix)
# print(result)

../GIS-Image-Stack-Processing/AOI/PK/Satellite_Tiles/
30m


In [18]:
raster = result['raster']
    
aoi_name = f"{country_code}"
    
crop_raster_rasterio(raster, points_within_raster, aoi_name, image_tile_suffix, image_tile_folder, tile_size=224, debug=False)
    

Crops are saved in ../GIS-Image-Stack-Processing/AOI/PK/Satellite_Tiles/


In [19]:
# Construct the VRT filename
vrt_file = f"../GIS-Image-Stack-Processing/AOI/{country_code}/{country_code}_Satellite_Tiles_{vrt_file_suffix}.vrt"
print(vrt_file)


../GIS-Image-Stack-Processing/AOI/PK/PK_Satellite_Tiles_30m.vrt


In [20]:
build_vrt(image_tile_folder, vrt_file)

VRT built successfully at ../GIS-Image-Stack-Processing/AOI/PK/PK_Satellite_Tiles_30m.vrt


/Users/billk/miniforge3/envs/py39-pt-test/lib/python3.9/site-packages/osgeo/gdal.py:312: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


In [21]:
print(vrt_file)

../GIS-Image-Stack-Processing/AOI/PK/PK_Satellite_Tiles_30m.vrt
